In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

In [2]:
# Download and unzip the dataset
data_path = "./data/"

# Load the data
ratings = pd.read_csv(data_path+'01_BX-Book-Ratings.csv', index_col=0)
users = pd.read_csv(data_path+'02_BX-Users.csv', index_col=0)
books = pd.read_csv(data_path+'03_BX-Books.csv', index_col=0)

C:\Users\user\AppData\Local\Temp\ipykernel_30688\3189584552.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(data_path+'03_BX-Books.csv', index_col=0)


In [3]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
# Preprocess the data
ratings = ratings.merge(books[['ISBN', 'Book-Title']], on='ISBN')
ratings = ratings.drop(['ISBN'], axis=1)

In [5]:
ratings.head()

,User-ID,Book-Rating,Book-Title
0,276725,0,Flesh Tones: A Novel
1,276726,5,Rites of Passage
2,276727,0,The Notebook
3,276729,3,Help!: Level 1
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...


In [6]:
ratings.shape

(1031136, 3)

In [7]:
users['User-ID'].nunique()

278858

In [8]:
ratings.head()

,User-ID,Book-Rating,Book-Title
0,276725,0,Flesh Tones: A Novel
1,276726,5,Rites of Passage
2,276727,0,The Notebook
3,276729,3,Help!: Level 1
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...


In [9]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
ratings['User-ID'].nunique()

92106

In [11]:
ratings['Book-Title'].nunique()

241071

In [12]:
books.ISBN.nunique()

271360

In [13]:
# Filter books with at least min_book_ratings ratings
min_book_ratings = 10
book_rating_counts = ratings['Book-Title'].value_counts()
filtered_books = book_rating_counts[book_rating_counts >= min_book_ratings].index
ratings = ratings[ratings['Book-Title'].isin(filtered_books)]

In [14]:
# Filter users who rated at least min_user_ratings books
min_user_ratings = 5
user_rating_counts = ratings['User-ID'].value_counts()
filtered_users = user_rating_counts[user_rating_counts >= min_user_ratings].index
ratings = ratings[ratings['User-ID'].isin(filtered_users)]

In [16]:
user_rating_counts

User-ID
11676     5827
35859     2741
153662    2409
76352     2132
198711    1837
          ... 
117173       1
117175       1
117176       1
117185       1
276721       1
Name: count, Length: 68518, dtype: int64

In [15]:
ratings.shape

(491897, 3)

In [17]:
# Create the user-item matrix
user_item_matrix = ratings.pivot_table(index='User-ID', columns='Book-Title', values='Book-Rating').fillna(0)

In [18]:
user_item_matrix.shape

(14322, 17446)

In [24]:
# Split user indices into train and test sets
user_indices = np.arange(user_item_matrix.shape[0])
train_user_indices, test_user_indices = train_test_split(user_indices, test_size=0.2, random_state=42)

In [25]:
def random_recommendation(ratings, n=10):
    unique_books = ratings['Book-Title'].unique()
    random_books = np.random.choice(unique_books, size=n, replace=False)
    return random_books

random_books = random_recommendation(ratings, n=10)
print("Random Score-Based Recommendations:")
for i, book in enumerate(random_books, 1):
    print(f"{i}. {book}")

Random Score-Based Recommendations:
1. If Morning Ever Comes : A Novel
2. Tommo &amp; Hawk
3. James Herriots Yorkshire
4. Kansas Troubles
5. Skybowl (Dragon Star, Book 3)
6. A Midwinter's Tale (O'Malley Novels (Forge Paperback))
7. Cock and Bull
8. House Of God.
9. The Drifter
10. Mondlaub.


In [26]:
def popularity_recommendation(ratings, n=10):
    popular_books = ratings.groupby('Book-Title')['Book-Rating'].count().sort_values(ascending=False).head(n).index
    return popular_books

popular_books = popularity_recommendation(ratings, n=10)
print("\nPopularity-Based Recommendations:")
for i, book in enumerate(popular_books, 1):
    print(f"{i}. {book}")


Popularity-Based Recommendations:
1. Wild Animus
2. The Lovely Bones: A Novel
3. The Da Vinci Code
4. The Nanny Diaries: A Novel
5. Bridget Jones's Diary
6. A Painted House
7. The Secret Life of Bees
8. Divine Secrets of the Ya-Ya Sisterhood: A Novel
9. Angels &amp; Demons
10. Life of Pi


In [27]:
def recommend_books(user_index, strategy, k=10):
    if strategy == "random":
        top_k_books = random_recommendation(ratings, n=k)
    elif strategy == "popularity":
        top_k_books = popularity_recommendation(ratings, n=k)
    else:
        raise ValueError("Invalid recommendation strategy")
    return top_k_books


In [28]:
def evaluate_model(strategy, k=10):
    true_positive = 0
    false_positive = 0
    false_negative = 0

    for user_index in test_user_indices:
        true_books = set(user_item_matrix.iloc[user_index][user_item_matrix.iloc[user_index] > 0].index)
        recommended_books = set(recommend_books(user_index, strategy, k))

        tp = len(true_books.intersection(recommended_books))
        fp = len(recommended_books - true_books)
        fn = len(true_books - recommended_books)

        true_positive += tp
        false_positive += fp
        false_negative += fn

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)

    return precision, recall

In [ ]:
# Evaluate the random score-based recommendation model
random_precision, random_recall = evaluate_model(strategy="random")
print(f"Random Score-Based: Precision = {random_precision:.4f}, Recall = {random_recall:.4f}")

In [ ]:
# Evaluate the popularity-based recommendation model
popularity_precision, popularity_recall = evaluate_model(strategy="popularity")
print(f"Popularity-Based: Precision = {popularity_precision:.4f}, Recall = {popularity_recall:.4f}")

In [18]:
users["User-ID"].nunique()

278858